In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
import os
for dirname, _, filenames in os.walk('./datas'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns ; sns.set()

from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report


from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import export_text

import warnings
warnings.filterwarnings('ignore')

## Lecture Donnee

In [ ]:
df = pd.read_csv('./datas/iris.csv')
df.head(10)

## Affichage Donnee

In [ ]:
df.info()

In [ ]:
df.describe()


In [ ]:
df = df.sample(frac=1)
df.head(10)

## Preparation Donnee

In [ ]:
x = df[["Id","SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]]
x.head(10)

In [ ]:
y = df[["Species"]]
y.head(10)

### Entrainement Modele

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

#scaler_X = StandardScaler() # Initialisation d’un standardiseur pour la variable explicative X
#scaler_y = StandardScaler() # Initialisation d’un standardiseur pour la variable cible y

# Apprentissage des paramètres (moyenne, écart-type) sur X_train et transformation des données
#X_train_scaled = scaler_X.fit_transform(X_train)

# Apprentissage des paramètres sur y_train et transformation des données
#y_train_scaled = scaler_y.fit_transform(y_train)

# Transformation de X_test à l’aide des paramètres appris sur X_train (pas de fit ici)
#X_test_scaled = scaler_X.transform(X_test)

# Transformation de y_test à l’aide des paramètres appris sur y_train (pas de fit ici non plus)
#y_test_scaled = scaler_y.transform(y_test)

### Fonction

In [ ]:
import joblib

In [ ]:
def save_model(model, scal_x, scal_y, filename: str):
    """
    Enregistre le modèle dans un fichier.
    
    Paramètres :
    - model : le modèle à enregistrer
    - filename : nom du fichier dans lequel enregistrer le modèle
    """
    filename = filename.lower()
    
    model_dir = "../models/"
    
    # Créer le répertoire models s'il n'existe pas
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # Créer le répertoire du modèle s'il n'existe pas
    model_path = os.path.join(model_dir, filename)
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    
    # Sauvegarder le modèle et les scalers
    joblib.dump(model, f"{model_path}/{filename}.pkl")
    joblib.dump(scal_x, f"{model_path}/{filename}_scaler_X.pkl")
    joblib.dump(scal_y, f"{model_path}/{filename}_scaler_y.pkl")
    print(f"Modèle enregistré dans {model_path}")

### Use DecisionTreeClassifier

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
y_pred = clf.predict([[128,6.1,3.0,4.9,1.8]])
print(y_pred)

In [ ]:
tree.plot_tree(clf)


### DecisionTreeClassifier Test

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test ,y_pred))

### DecisionTreeClassifier Save

In [ ]:
save_model(clf, X_train, y_train, "decision_tree_classifier")

### Use RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Création et entraînement du modèle RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf = rf_clf.fit(X_train, y_train.values.ravel())

# Prédiction sur un exemple
y_pred_rf = rf_clf.predict([[128,6.1,3.0,4.9,1.8]])
print(f"Prédiction RandomForest: {y_pred_rf}")

In [ ]:
# Affichage de l'importance des features
importances = rf_clf.feature_importances_
feature_names = X_train.columns

# Visualisation de l'importance des features
plt.figure(figsize=(10, 6))
plt.bar(feature_names, importances)
plt.title('Importance des features - RandomForestClassifier')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

### Use RandomForestClassifier Test

In [ ]:
# Test du RandomForestClassifier sur les données de test
y_pred_rf_test = rf_clf.predict(X_test)

# Affichage du rapport de classification
print("Rapport de classification - RandomForestClassifier:")
print(classification_report(y_test, y_pred_rf_test))

# Matrice de confusion
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred_rf_test)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf_clf.classes_)
disp.plot()
plt.title('Matrice de confusion - RandomForestClassifier')
plt.show()

### RandomForestClassifier Save

In [ ]:
# Sauvegarder le modèle RandomForestClassifier
save_model(rf_clf, X_train, y_train, "random_forest_classifier")

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import export_text
from sklearn.ensemble import RandomForestClassifier

import joblib
import warnings
warnings.filterwarnings('ignore')

print("Bibliothèques importées avec succès")

In [ ]:
# Test de la fonction save_model corrigée
def save_model(model, scal_x, scal_y, filename: str):
    """
    Enregistre le modèle dans un fichier.
    
    Paramètres :
    - model : le modèle à enregistrer
    - filename : nom du fichier dans lequel enregistrer le modèle
    """
    filename = filename.lower()
    
    model_dir = "../models/"
    
    # Créer le répertoire models s'il n'existe pas
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    
    # Créer le répertoire du modèle s'il n'existe pas
    model_path = os.path.join(model_dir, filename)
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    
    # Sauvegarder le modèle et les scalers
    joblib.dump(model, f"{model_path}/{filename}.pkl")
    joblib.dump(scal_x, f"{model_path}/{filename}_scaler_X.pkl")
    joblib.dump(scal_y, f"{model_path}/{filename}_scaler_y.pkl")
    print(f"Modèle enregistré dans {model_path}")

# Test de la fonction avec un modèle simple
from sklearn.tree import DecisionTreeClassifier
df = pd.read_csv('./datas/iris.csv')
x = df[["Id","SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"]]
y = df[["Species"]]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

clf = DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

# Test de sauvegarde
save_model(clf, X_train, y_train, "test_model")
print("Test de la fonction save_model réussi")